In [3]:
import pandas as pd
from io import StringIO
import numpy as np

In [47]:
csv_data = '''a, b, c, d
1, 2, 3, 4
5, 6,, 8
0, 11, 12,'''

In [48]:
csv_data

'a, b, c, d\n1, 2, 3, 4\n5, 6,, 8\n10, 11, 12,'

In [49]:
df = pd.read_csv(StringIO(csv_data))

In [22]:
df

,a,b,c,d
0,1,2,3.0,4.0
1,5,6,NaN,8.0
2,0,11,12.0,NaN


In [23]:
df.isnull().sum()   ### 각 열에 대한 결측값 확인하기

a     0
 b    0
 c    1
 d    1
dtype: int64

In [24]:
df.isnull().sum(axis = 1) ### 각 행에 대한 결측값 확인하기

0    0
1    1
2    1
dtype: int64

In [27]:
df.values    ### 사이킷 런은 numpy 형식으로 작업하도록 개발되었기 때문에 df.values를 이용하여 numpy 형식으로 반환받을 수 있다.

array([[  1.,   2.,   3.,   4.],
       [  5.,   6.,  nan,   8.],
       [  0.,  11.,  12.,  nan]])

In [28]:
df.dropna() ### dropna 메서드를 이용하여 결측치가 있는 행(axis = 0) 또는 열(axis = 1)을 제거할 수 있다.

,a,b,c,d
0,1,2,3.0,4.0


In [29]:
df.dropna(how = 'all')   ### 모든 열이 결측치인 행 제거

,a,b,c,d
0,1,2,3.0,4.0
1,5,6,NaN,8.0
2,0,11,12.0,NaN


In [30]:
df.dropna(thresh = 4)  # 결측치가 아닌 값이 4개보다 적은 행 제거

,a,b,c,d
0,1,2,3.0,4.0


In [37]:
df.dropna(subset = ['d'])   ### 'C' 열에 결측치가 있는 경우에만 해당 행 제거

KeyError: ['d']

In [39]:
########## 결측치 보정하기(Impute) ##########

In [40]:
from sklearn.preprocessing import Imputer

In [42]:
imr = Imputer(missing_values = 'NaN',   ### 결측치는 'NaN'으로 코딩되어 있음
             strategy = 'mean',    #### 결측치를 평균으로 대처(median, most_frequent 등도 사용 가능)
             axis = 0)         #### 기준은 해당 열(axis = 1일 경우 행)

In [50]:
imr = imr.fit(df)
imputed_data = imr.transform(df.values)
imputed_data

array([[  1. ,   2. ,   3. ,   4. ],
       [  5. ,   6. ,   7.5,   8. ],
       [ 10. ,  11. ,  12. ,   6. ]])

In [46]:
df

,a,b,c,d
0,1,2,3.0,4.0
1,5,6,NaN,8.0
2,0,11,12.0,NaN


In [51]:
########## 범주형 데이터 다루기 ##########

In [52]:
df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'calss2'],
                   ['blue', 'XL', 15.3, 'class1']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,calss2
2,blue,XL,15.3,class1


In [53]:
size_mapping = {    ### 순서형 변수(size) 매핑하기
    'XL' : 3,
    'L' : 2,
    'M' : 1
}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,calss2
2,blue,3,15.3,class1


In [60]:
######## 분류 레이블 인코딩 ######## (class 변수)

In [61]:
class_mapping = {label:idx for idx, label in
                enumerate(np.unique(df['classlabel']))}
class_mapping

{'calss2': 0, 'class1': 1}

In [62]:
df['classlabel'] = df['classlabel'].map(class_mapping)     ### 매핑 dict를 이용하여 분류 레이블 정수로 변환

In [63]:
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [64]:
inv_class_mapping = {v : k for k, v in class_mapping.items()}   ### 매핑 dict를 이용하여 분류 레이블을 string으로 변환
df['classlabel'] = df['classlabel'].map(inv_class_mapping)

In [65]:
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,calss2
2,blue,3,15.3,class1


In [78]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1], dtype=int64)

In [66]:
############# 명목형 척도에 one-hot-encoding 하기 #############

In [79]:
x = df[['color', 'size', 'price']].values
color_le = LabelEncoder()

In [68]:
x

array([['green', 1, 10.1],
       ['red', 2, 13.5],
       ['blue', 3, 15.3]], dtype=object)

In [82]:
x[:, 0] = color_le.fit_transform(x[:, 0])   ### 이 상태에서 코딩을 진행하면 green이 blue보다 크고, red가 green보다 크다고 인식함.

In [83]:
x

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In [84]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features = [0])   ##### categoriacal_features 옵션을 통해 더미와하려는 변수의 열 위치를 입력
ohe.fit_transform(x).toarray()

array([[  0. ,   1. ,   0. ,   1. ,  10.1],
       [  0. ,   0. ,   1. ,   2. ,  13.5],
       [  1. ,   0. ,   0. ,   3. ,  15.3]])

In [87]:
pd.get_dummies(df[['price', 'color', 'size']])    ### 혹은 pandas의 get_dummies를 이용하여 간단하게 변환할 수도 있다.

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [88]:
############ 데이터를 훈련용과 테스트용으로 분할하기 ############

In [7]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header = None)
df_wine.columns = ['Class label', 'Alcohol',
                  'Malice acid', 'Ash',
                  'Alcalinity of ash', 'Magnesium',
                  'Total phenols', 'Flavanoids',
                  'Nonflavanoid phenols', 'Proanthocyanins',
                  'Colro intensity', 'Hue',
                  'OD280/OD315 of diluted wines',
                  'Proline']
print('Class labels', np.unique(df_wine['Class label']))

Class labels [1 2 3]


In [8]:
df_wine.head()

,Class label,Alcohol,Malice acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Colro intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [9]:
from sklearn.cross_validation import train_test_split
x, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values    ### iloc 메소드는 정수형 위치형으로 데이터 찾기. lable로 찾는 것은 loc 메소드
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [95]:
################ 정규화 하기 ################

In [5]:
from sklearn.preprocessing import MinMaxScaler   ### minmax가 아닌 표준화는 StandardScaler를 사용한다.

In [11]:
mms = MinMaxScaler()
x_train_norm = mms.fit_transform(x_train)
x_test_norm = mms.transform(x_test)

In [13]:
print(x_train_norm, x_test_norm)

[[ 0.72043011  0.20378151  0.53763441 ...,  0.48717949  1.          0.5854251 ]
 [ 0.31989247  0.08403361  0.31182796 ...,  0.27350427  0.64102564  0.        ]
 [ 0.60215054  0.71218487  0.48387097 ...,  0.04273504  0.10622711
   0.42348178]
 ..., 
 [ 0.37365591  0.1512605   0.44623656 ...,  0.44444444  0.61904762
   0.02672065]
 [ 0.77150538  0.16596639  0.40860215 ...,  0.31623932  0.75457875
   0.54493927]
 [ 0.84139785  0.34033613  0.60215054 ...,  0.06837607  0.16117216
   0.28178138]] [[ 0.72849462  0.16386555  0.47849462  0.29896907  0.52173913  0.53956835
   0.74311927  0.13461538  0.37974684  0.4364852   0.32478632  0.70695971
   0.60566802]
 [ 0.47311828  0.37394958  0.60215054  0.58762887  0.45652174  0.13669065
   0.27217125  0.19230769  0.26582278  0.90926457 -0.05128205  0.07326007
   0.13603239]
 [ 0.36021505  0.05042017  0.43010753  0.43298969  0.18478261  0.86330935
   0.80428135  0.09615385  0.45886076  0.30276982  0.58119658  0.58608059
   0.08744939]
 [ 0.68010753  